# ADVARSEL: NÅR ANIMATIONEN ER FÆRDIG SKAL KODEN I BUNDEN KOMMENTERES UD OG CELLEN KØRES IGEN FØR DER GEMMES. ELLERS GEMMER DU EN FIL PÅ 100+ MB!!!

In [75]:
import numpy as np

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm.notebook import tqdm

In [76]:
# # Parameters
# s = 10
# r = 28
# b = 2.667

# Colours
BACKGROUND_COLOR = "black"
PLOT_COLOR = "grey"

# Width of the plotted trajectory
LW = 1

# Camera rotation speed
ROTATION_SPEED = 1/2

In [77]:
def get_fixed_points(s, r, b):
    '''Returns a list containing the fixed points of the system, given the parameter values.'''
    return [[0, 0, 0], [np.sqrt(b*(r-1)), np.sqrt(b*(r-1)), r-1], [-np.sqrt(b*(r-1)), -np.sqrt(b*(r-1)), r-1]]

def get_initial_states(initial_states=None, num_states=3):
    '''initial_states must be a nested list of initial states i.e. [[x_0, y_0, z_0], [x_1, y_1, z_1], ...]

    num_states must match the length of initial_states
    '''
    if initial_states == None:
        return [np.random.uniform(low=-10, high=10, size=(3,)) for i in range(num_states)] # If no initial state was given, we return num_states random states
    else:
        return initial_states

def calculate_time_steps(dt, total_duration):
    '''Calculates how many time steps are needed to simulate the trajectory for total_duration seconds.'''
    n_time_steps = int(np.floor(total_duration / dt))

    return n_time_steps

def lorenz(r_vec, s, r, b):
    '''Implements the Lorenz equations.'''
    x, y, z = r_vec
    x_dot = s * (y - x)
    y_dot = r * x - y - x * z
    z_dot = x * y - b * z

    return np.array([x_dot, y_dot, z_dot])

def RK4(r_vec, s, r, b, dt):
    '''Numerical integration with the fourth order Runge-Kutta method. Returns the new position, given the old one and the parameter values of the system.'''
    k1 = lorenz(r_vec, s, r, b) * dt
    k2 = lorenz(r_vec + 1/2 * k1, s, r, b) * dt
    k3 = lorenz(r_vec + 1/2 * k2, s, r, b) * dt
    k4 = lorenz(r_vec + k3, s, r, b) * dt

    return r_vec + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

def calculate_trajectories(initial_states, n_time_steps, s, r, b, dt):
    '''Calculates the trajectories from each of the initial conditions. 
    
    Returns a list containing the trajectories. Each trajectory is itself a n_time_steps x 3 list (there are three coordinates and n_time_step iterations...)'''
    trajectories = []

    for r_vec_initial in initial_states:
        pos = np.zeros((n_time_steps, 3))
        pos[0] = r_vec_initial
        for i in range(n_time_steps - 1):
            pos[i+1] = RK4(pos[i], s, r, b, dt)
        trajectories.append(pos)

    return trajectories

def update_plot(ax, trajectories, fp_arr, i, colors):
    '''Updates the plot for the animation. Colors is an array of the colors associated with each trajectory.
    
    A figure and subplot must be defined in advance and passed to this function, see main().'''
    try:
        ax.view_init(0, -56 + i * ROTATION_SPEED) # Rotates the camera around the z-axis
        ax.clear()
        ax.scatter(fp_arr[0][0], fp_arr[0][1], fp_arr[0][2], color="red", label="Fixed Point #1")
        ax.scatter(fp_arr[1][0], fp_arr[1][1], fp_arr[1][2], color="lime", label="Fixed Point #2")
        ax.scatter(fp_arr[2][0], fp_arr[2][1], fp_arr[2][2], color="blue", label="Fixed Point #3")
        ax.set(facecolor=BACKGROUND_COLOR)
        ax.set_axis_off()
        for pos, color in zip(trajectories, colors):
            if i < len(pos):
                ax.plot(pos[:i, 0], pos[:i, 1], pos[:i, 2], color=color, linewidth=LW)
                ax.scatter(pos[i, 0], pos[i, 1], pos[i, 2], color="white", linewidth=LW, marker="*")
            else:
                print(f"Index {i} is out of bounds for trajectory with length {len(pos)}")
        ax.legend()
    except Exception as e:
        print(f"Error updating plot for frame {i}: {e}")



In [ ]:
def main(initial_states, num_states, colors, total_duration=5, s=10, r=28, b=8/3, dt=0.01):
    fp_arr = get_fixed_points(s, r, b)
    initial_states = get_initial_states(initial_states=initial_states, num_states=num_states)
    n_time_steps = calculate_time_steps(dt, total_duration)
    trajectories = calculate_trajectories(initial_states, n_time_steps, s, r, b, dt)

    fig = plt.figure(facecolor=BACKGROUND_COLOR, figsize=(10, 10))
    ax = fig.add_subplot(111, projection="3d")

    ani = animation.FuncAnimation(fig, lambda i: update_plot(ax, trajectories, fp_arr, i, colors), frames=tqdm(range(n_time_steps)), repeat=True)
    
    plt.close()

    print("The initial states were:")
    for i in range(len(initial_states)):
        print(f"{initial_states[i]} with the color {colors[i]}")

    return HTML(ani.to_jshtml())

colorlist = ["red", "green", "blue", "yellow", "white", "cyan", "orange", "purple", "pink", "brown", "grey", "olive", "teal", "navy", "maroon", "silver", "gold", "indigo", "crimson", "darkorange"] # 20 colors add more if need be

#ani = main([[0, 1, 10], [0, 1, 10.1], [0, 1, 10.2]], num_states=3, colors=colorlist[:3], total_duration=15)



  0%|          | 0/1500 [00:00<?, ?it/s]

The initial states were:
[0, 1, 10] with the color red
[0, 1, 10.1] with the color green
[0, 1, 10.2] with the color blue


In [ ]:
# ani